In [1]:
include("src/JuTrack.jl")

Main.JuTrack

In [2]:
using .JuTrack
using Enzyme
using Test
using BenchmarkTools
include("test/ssrf_ring.jl")

ssrf (generic function with 1 method)

In [21]:
#CTPS = CTPS (intial value, what palce in array that derv occupies, # of unique variables finding derv of, order of calculation)

In [59]:
function fodo_ring(K1, RAD)
        D1 = DRIFT(name="D1", len=0.34)
        D2 = DRIFT(name="D2", len=0.59)
        QF1 = KQUAD(name="QF1", len=0.32, k1=K1, rad=RAD )
        QD1 = KQUAD(name="QD1", len=0.32, k1=-1.192160, rad=RAD  )
        BendingAngle = pi/20
        BD1 = SBEND(name="BD1", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=RAD )
        BD2 = SBEND(name="BD2", len=0.72, angle=BendingAngle/2, e1=0.0, e2=BendingAngle/2, rad=RAD  )

        FODO = (QF1, D1, QD1, D1, QF1)
        BD1_comp = (BD1)
        BD2_comp = (BD2)
        D2_comp = (D2)

        CELL = Vector{AbstractElement}(undef, 8)
        for i in eachindex(FODO)
                CELL[i] = FODO[i]
        end
        CELL[length(FODO)+1] = BD1
        CELL[length(FODO)+2] = D2
        CELL[length(FODO)+3] = BD2
                # CELL[i+172] = M5[i]
        
        ELIST = Vector{AbstractElement}(undef, 4*length(CELL))
        for i in eachindex(CELL)
                ELIST[i] = CELL[i]
                ELIST[i+length(CELL)] = CELL[i]
                ELIST[i+2*length(CELL)] = CELL[i]
                ELIST[i+3*length(CELL)] = CELL[i]
        end
        return ELIST
end

function track(k1_in)
        particles = [0.001 0.0001 0.0005 0.0002 0.0 0.0; 0.001 0.0 0.0 0.0 0.0 0.0] #There is 2 particles here, and their 6 vector
        beam = Beam(particles)
        line = fodo_ring(k1_in[1],0.0)
        linepass!(line, beam)
        return beam.r
end

function track_TPS(k1_in)
        line = fodo_ring(k1_in[1],0.0)
        x = CTPS(0.0, 1, 6, 3)
        xp = CTPS(0.0, 2, 6, 3)
        y = CTPS(0.0, 3, 6, 3)
        yp = CTPS(0.0, 4, 6, 3)
        delta = CTPS(0.0, 5, 6, 3)
        z = CTPS(0.0, 6, 6, 3)
        rin = [x, xp, y, yp, delta, z]
        linepass_TPSA!(line, rin)
        return rin
end

#Between track() and track_TPS(), it seems I am only able to have one particle through TPS? Create function to loop through multiple? 

function twiss(k1_in)
        fodo = fodo_ring(k1_in[1],0)
        twiss_in = EdwardsTengTwiss(betax=2.0, betay=3.0)
        twiss_out = Twissline(twiss_in, fodo, 0.0, 1, length(fodo) )[3] #(Twiss In, ring/line, dp, order, end index)
        return twiss_out
end

twiss (generic function with 1 method)

In [60]:
# x = CTPS(3.0, 1, 6, 3)
# y = CTPS(2.0, 2, 6, 3)
# println(y.map[1])

println(twiss([-1.063]))

EdwardsTengTwiss(6.308037258524679e6, 6.0056676347256435, -1.9074882104729326e6, -2.7408511462377265, 576805.6090945166, 1.4173719765332176, 93.76029383266201, 28.48046008080599, 0.0, 0.0, 0.571626821312843, 2.75186009043917, 0.5410009564987482, 0.8410219765662722, 0.3799410467651746, -0.9250107031721219, [0.0 0.0; 0.0 0.0], 1)


In [40]:
# track(-1)
track_TPS(-1)

6-element Vector{CTPS{Float64, 6, 3}}:
 CTPS{Float64, 6, 3}(3, 84, [0.0, 1288.1315606600112, 1701.0475715953958, 0.0, 0.0, 0.0, 85.58044593245963, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 655.532789287764], Base.RefValue{PolyMap}(PolyMap(6, 3, [0 0 … 0 0; 1 1 … 0 0; … ; 3 0 … 1 0; 3 0 … 0 0])))
 CTPS{Float64, 6, 3}(3, 84, [0.0, 385.551562025798, 509.1425195519279, 0.0, 0.0, 0.0, 25.744816423742847, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 128.99555750358232], Base.RefValue{PolyMap}(PolyMap(6, 3, [0 0 … 0 0; 1 1 … 0 0; … ; 3 0 … 1 0; 3 0 … 0 0])))
 CTPS{Float64, 6, 3}(3, 84, [0.0, 0.0, 0.0, -1.2053179593863022, 1.8246766866199848, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, -29.708565451139986, 0.0, 0.0, 0.0, 0.0], Base.RefValue{PolyMap}(PolyMap(6, 3, [0 0 … 0 0; 1 1 … 0 0; … ; 3 0 … 1 0; 3 0 … 0 0])))
 CTPS{Float64, 6, 3}(3, 84, [0.0, 0.0, 0.0, -0.7376050907354532, 0.2869705958528243, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0

In [18]:

# println(track_TPS(-1.063770))
# println(length(track_TPS(-1.063770)))
# x = [-1.063770]
# println(track_TPS(x))
# @btime track_TPS(x) 
# @btime grad = gradient(Forward, test_track_TPS, x)

In [19]:

# println(track(-1.063770))
# x = [-1.063770]
# println(track(x))
# @btime track(x) 
# @btime grad = gradient(Forward, track, x)
# println(gradient(Forward, track, x))